In [100]:
#Sid Cox spring 2020

#tool to get the orfs: 
#ORFfinder -g 2 -s 1 -ml 150 -n true -in human_tf.fasta -out human_tf_orfs.fasta -outfmt 0
# conda install nomkl numpy scipy scikit-learn numexpr
# conda install -c bioconda hmmer 
# hmmscan --domtblout human_tf_domains.txt hmmdir/Pfam-A.hmm covid-detector/human_tf_cds.fasta 

#covid-detector data structure
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #basic plotting is good enough here
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

#load up the spreadsheet
#df = pd.read_excel("covid-detector/Human\ Transcription\ Factors.xlsx ")

df = pd.read_excel("DNA_TF_GO_prot.xlsx")

In [101]:
from Bio import SearchIO
for qresult in SearchIO.parse('human_tf_domains.txt', 'hmmscan3-domtab'):
    qresult.id

In [65]:
def read_hmmscan(path, set_index="query_name", sort_values="sequence|_|e-value"):
    # Build pd.DataFrame
    df_tmp = pd.read_table(path, header=None, index_col=0)
    mask_idx = df_tmp.index.map(lambda x: x.startswith("#") == False)
    tmp_list = df_tmp.loc[mask_idx,:].index.map(lambda x: [y for y in x.split(" ") if (len(y) > 0) and (y != "-")]).tolist()
    data = list()
    for line in tmp_list:
        data.append(pd.Series(line))
    df_hmmscan = pd.DataFrame(data)
    # Labels
    id_cols = ["target_name", "query_name"]
    sequence_cols = ["sequence|_|e-value", "sequence|_|score", "sequence|_|bias"]
    domain_cols = ["domain|_|e-value", "domain|_|score", "domain|_|bias"]
    misc_cols = ["exp", "reg", "clu", "ov", "env", "dom", "rep", "inc"]
    df_hmmscan.columns  = id_cols + sequence_cols + domain_cols + misc_cols
    # Force types
    df_hmmscan.loc[:,sequence_cols + domain_cols] = df_hmmscan.loc[:,sequence_cols + domain_cols].astype(float)
    df_hmmscan.loc[:,"exp"] = df_hmmscan.loc[:,"exp"].astype(float)
    df_hmmscan.loc[:,misc_cols[1:]] = df_hmmscan.loc[:,misc_cols[1:]].astype(int)
    # Reformat structure
    if set_index:
        df_hmmscan = df_hmmscan.set_index(set_index)
    if sort_values:
        df_hmmscan = df_hmmscan.sort_values(sort_values, ascending=True)
    return df_hmmscan
df2 = read_hmmscan("human_tf_domains.txt")

In [ ]:
#pull in the fasta
short_sequences = [] # Setup an empty list

seqfile = "my_fasta.txt"
count =1
for record in SeqIO.parse(seqfile, "fasta"):
    #print(record.id)
    count=count+1
    coding_dna = record.seq
    protein = coding_dna.translate(table="Vertebrate Mitochondrial")
    pr_tostop1 = coding_dna.translate(to_stop=True,table="Vertebrate Mitochondrial")
    pr_tostop2 = coding_dna[1:].translate(to_stop=True,table="Vertebrate Mitochondrial")
    pr_tostop3 = coding_dna[2:].translate(to_stop=True,table="Vertebrate Mitochondrial")

    #print(protein)
    print(pr_tostop1)
    print(pr_tostop2)
    print(pr_tostop3)
    print("")

    if len(record.seq) < 3000 :
        # Add this record to our list
        short_sequences.append(record)
#print(count)

print("Found %i short sequences" % len(short_sequences))
SeqIO.write(short_sequences, "short_seqs.fasta", "fasta")

In [126]:
df = pd.read_excel("DNA_TF_GO_prot.xlsx", sheet_name = "DNA_TF_GO_Prot_toronto")
df['translated_sequence']
df['protein_sequence']

#loop through all the sequences and build a fasta file
coding_dna = []
cds = []
ofile = open("human_tf_cds.fasta", "w")
for i in range(len(df['ENSG_ENST'])):
    coding_dna.append(df['translated_sequence'][i])
    cds.append(df['protein_sequence'][i])
    if isinstance(df['protein_sequence'][i], str): #is the sequence a string or empty
        ofile.write(df['ENSG_ENST'][i] + "\n" + df['protein_sequence'][i] + "\n")
ofile.close() #do not forget to close it
#write out to fasta
ofile = open("human_tf_orf.fasta", "w")
for i in range(len(df['ENSG_ENST'])):
    if isinstance(df['translated_sequence'][i], str): #is the sequence a string or empty
        ofile.write(df['ENSG_ENST'][i] + "\n" + df['translated_sequence'][i] + "\n")
ofile.close() #do not forget to close it

#its easier to write out to fasta then import into seqrecord than to build one
cdsfile = "human_tf_cds.fasta"
orffile = "human_tf_orf.fasta"
count =1
for (orfrecord, cdsrecord) in zip(SeqIO.parse(orffile, "fasta"),SeqIO.parse(cdsfile, "fasta")):
    count=count+1
    coding_dna = orfrecord.seq
    protein = coding_dna.translate(table="Vertebrate Mitochondrial")

In [132]:
cdsrecord.seq
orfrecord.seq.translate(table="Vertebrate Mitochondrial")
    #something isn't lining up between the ORF and CDS

Seq('MET*PTALMSSTVAAAAPAAGAAS*KESPG*WGLGEDPTGVSPSLQCRVCGDSS...V*W', HasStopCodon(ExtendedIUPACProtein(), '*'))

In [ ]:
class Transcript:
    #primary transcripts of human TFs, will contain multiple isoforms and proteins    
    def __init__(self, name):
        self.name = name

class Isoform:
    #an isoform derived from a primary transcript, will generall contain one or zero proteins
    def __init__(self, name):
        self.name = name

class TF:
    #a protein sequence of a transcription factor
    def __init__(self, name, dna):
        self.name = name
        self.dna = dna #the coding dna sequence
        self.aa = "" #the amino acid sequence of this isoform
        self.dbd = "" #the dna binding domain